In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import PIL.Image
# import vgg1
import download
import imagecomputations
import lossfunction
import requests

In [2]:
import vgg1

In [3]:
vgg1.mdownload()

Data has already been downloaded


In [4]:
def mse(a, b):
    sqr = tf.square(a - b)
    redmean = tf.reduce_mean(sqr)
    return redmean

	
def create_content_loss(session, model, content_image, ids):
    feed_dict = model.create_feed_dict(image=content_image)

    layers = model.get_layer_tensors(ids)

    content_values = session.run(layers, feed_dict=feed_dict)
#     filewriter = tf.summary.FileWriter('C:\\Users\\Raj Mehta\\Documents\\GitHub\\Neural Style Transfer', values.graph)

    with model.graph.as_default():
        layer_losses = []
    
        for value, layer in zip(content_values, layers):
            loss = mse(layer, value)
#             filewriter = tf.summary.FileWriter('C:\\Users\\Raj Mehta\\Documents\\GitHub\\nst', loss.graph)

            layer_losses.append(loss)

        total_loss = tf.reduce_mean(layer_losses)
        
    return total_loss

def gram_matrix(tensor):
    shape = tensor.get_shape()
    
    num_channels = int(shape[3])

    matrix = tf.reshape(tensor, shape=[-1, num_channels])
    
    
    gram = tf.matmul(tf.transpose(matrix), matrix)

    return gram

def create_style_loss(session, model, style_image, ids):
    feed_dict = model.create_feed_dict(image=style_image)

    layers = model.get_layer_tensors(ids)

    with model.graph.as_default():
        gram_layers = [gram_matrix(layer) for layer in layers]

        values = session.run(gram_layers, feed_dict=feed_dict)

        layer_losses = []
    
        for value, gram_layer in zip(values, gram_layers):
#             value_const = tf.constant(value)

            loss = mse(gram_layer, value)

            layer_losses.append(loss)

        total_loss = tf.reduce_mean(layer_losses)
        
    return total_loss


In [30]:
def style_transfer(content_image, style_image,
                   content_layer_ids, style_layer_ids,
                   weight_content=1.5, weight_style=10.0,
                   weight_denoise=0.3,
                   num_iterations=120, step_size=10.0):
    model = vgg1.VGG16()

    session = tf.InteractiveSession(graph=model.graph)

    print("Content layers:")
    print(model.get_lnames(content_layer_ids))
    print()

    print("Style layers:")
    print(model.get_lnames(style_layer_ids))
    print()

    loss_content = create_content_loss(session=session,
                                       model=model,
                                       content_image=content_image,
                                       ids=content_layer_ids)

    loss_style = create_style_loss(session=session,
                                   model=model,
                                   style_image=style_image,
                                   ids=style_layer_ids) 
    
    con = tf.Variable(loss_content, name='con')
    con1 = tf.Variable(loss_style, name='con1')
#     adj_content = tf.Variable(1/1e-10, name='adj_content')
#     adj_style = tf.Variable(1/1e-10, name='adj_style')

#     session.run([adj_content.initializer,
#                  adj_style.initializer])

#     update_adj_content = adj_content.assign(1.0 / (loss_content))
#     update_adj_style = adj_style.assign(1.0 / (loss_style))

#     update_adj_content = adj_content.assign(1.0 / 0.001)
#     update_adj_style = adj_style.assign(1.0 / 0.001)
#     opt = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
    loss_combined = weight_content *  con + \
                    weight_style *  con1
#     opt = tf.train.AdamOptimizer(1e-4).compute_gradients(loss_combined, model.input)
#     init_op = tf.initialize_all_variables()
#     gradi = init_op.compute_gradients(loss_combined)
#     appgrad = opt.apply_gradients(gradi)

    gradient = tf.gradients(loss_combined, model.input)

#     run_list = [init_op, update_adj_content, update_adj_style]

    mixed_image = np.random.rand(*content_image.shape) + 128
    
#     adj_image = tf.Variable(tf.convert_to_tensor(mixed_image), name='adj_image')
    
#     ad = tf.Variable(loss_combined, name='ad')
    opt = tf.train.AdamOptimizer(1e-4).minimize(loss_combined)
    init_op = tf.global_variables_initializer()
    run_list = [gradient]
#     , update_adj_content, update_adj_style]


    for i in range(num_iterations):
        feed_dict = model.create_feed_dict(image=mixed_image)
#         session.run(init_op)

#         grad, adj_content_val, adj_style_val =  \
        grad = session.run(run_list, feed_dict=feed_dict)
#         session.run(init_op)
#         session.run(opt, feed_dict = model.input)
        
#         filewriter = tf.summary.FileWriter('C:\\Users\\Raj Mehta\\Documents\\GitHub\\nst', grad.graph)


        grad = np.squeeze(grad)

        step_size_scaled = step_size / (np.std(grad) + 1e-8)

        mixed_image -= grad * step_size_scaled
        session.run(init_op)
        session.run(opt, feed_dict = feed_dict)
#         session.run(appgrad)

        mixed_image = np.clip(mixed_image, 0.0, 255.0)

        print(". ", end="")

        if (i % 10 == 0) or (i == num_iterations - 1):
            print()
            print("Iteration:", i)

            msg = "Weight Adj. for Content: {0:.2e}, Style: {1:.2e}"
#             print(msg.format(adj_content_val, adj_style_val))

            imagecomputations.plot_images(content_image=content_image,
                        style_image=style_image,
                        mixed_image=mixed_image)
            
    print()
    print("Final image:")
    imagecomputations.plot_final_image(mixed_image)

    session.close()
    
    return mixed_image

In [31]:
content_filename = 'images/elon_musk.jpg'
content_image = imagecomputations.get_image(content_filename)

In [32]:
style_filename = 'images/style8.jpg'
style_image = imagecomputations.get_image(style_filename)

In [33]:
content_layer_ids = [4]

In [34]:
style_layer_ids = list(range(13))

In [ ]:
%%time
img = style_transfer(content_image=content_image,
                     style_image=style_image,
                     content_layer_ids=content_layer_ids,
                     style_layer_ids=style_layer_ids,
                     weight_content=3.0,
                     weight_style=10.0,
                     weight_denoise=0.3,
                     num_iterations=30,
                     step_size=10.0)